In [20]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.corpus import wordnet

In [156]:
train=pd.read_csv('mask_train2.csv')
test=pd.read_csv('mask_test2.csv')
#주의: baseline에서는 validation_sample을 사용하지 않았습니다.

In [27]:
del train['full_log']
del test['full_log']

In [157]:
train_text=list(train['first'])
train_level=np.array(train['level'])

In [158]:
train

,id,level,full_log,pre_log,first
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err...",Sep localhost kibana type error timestamp Z ta...,Sep localhost kibana type error timestamp Z ta...
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...,Feb localhost logstash INFO logstash outputs e...,Feb localhost logstash INFO logstash outputs e...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,Jan localhost kibana type error timestamp Z ta...
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,Jan localhost kibana type error timestamp Z ta...
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...,type SYSCALL msg audit arch syscall success ye...,type SYSCALL msg audit arch syscall success ye...
...,...,...,...,...,...
472967,472967,0,Feb 28 10:10:06 localhost logstash: 7738 error:,Feb localhost logstash error,Feb localhost logstash error
472968,472968,1,type=SYSCALL msg=audit(1611890993.458:321827):...,type SYSCALL msg audit arch syscall success ye...,type SYSCALL msg audit arch syscall success ye...
472969,472969,0,"Oct 12 02:20:29 localhost kibana: {""type"":""log...",Oct localhost kibana type log timestamp Z tags...,Oct localhost kibana type log timestamp Z tags...
472970,472970,0,"Jan 15 09:02:43 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,Jan localhost kibana type error timestamp Z ta...


In [159]:
#CountVectorizer로 벡터화

#TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=CountVectorizer(analyzer="word", max_features=30000)
#vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)

train_features=vectorizer.fit_transform(train_text)

In [160]:
TEST_SIZE=0.2
RANDOM_SEED=42

train_x, eval_x, train_y, eval_y=train_test_split(train_features, train_level, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [161]:
#랜덤포레스트로 모델링
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_estimators=100)

forest.fit(train_x, train_y)

RandomForestClassifier()

In [162]:
forest.score(eval_x, eval_y)


0.997864580580369

In [163]:
#crosstab으로 확인
pred=forest.predict(eval_x)
crosstab = pd.crosstab(eval_y, pred, rownames=['real'], colnames=['pred'])
crosstab

pred,0,1,2,3,4,5
real,,,,,,
0,66725,59,0,0,0,2
1,120,26426,0,0,0,1
2,0,0,2,0,0,0
3,1,1,0,799,0,0
4,0,0,0,0,3,0
5,12,3,0,0,0,438
6,1,0,0,2,0,0


In [16]:
preds=forest.predict(eval_x)
probas=forest.predict_proba(eval_x)
print(preds.shape)
print(probas.shape)

(94595,)
(94595, 7)


In [17]:
preds[np.where(np.max(probas, axis=1)<0.6)] = 7

preds[np.where((np.max(probas, axis=1)<0.4) & (preds == 0))=7
preds[np.where((np.max(probas, axis=1)<0.4) & (preds == 1))=7
preds[np.where((np.max(probas, axis=1)<0.9) & (preds == 2))=7
preds[np.where((np.max(probas, axis=1)<0.7) & (preds == 3))=7
preds[np.where((np.max(probas, axis=1)<0.9) & (preds == 4))=7
preds[np.where((np.max(probas, axis=1)<0.95) & (preds == 5))=7
preds[np.where((np.max(probas, axis=1)<0.8) & (preds == 6))=7


In [18]:

new_crosstab = pd.crosstab(eval_y, preds, rownames=['real'], colnames=['pred'])
new_crosstab

pred,0,1,2,3,4,5,7
real,,,,,,,
0,66682,40,0,0,0,0,64
1,102,26422,0,0,0,1,22
2,0,0,2,0,0,0,0
3,1,1,0,799,0,0,0
4,0,0,0,0,3,0,0
5,11,2,0,0,0,437,3
6,1,0,0,0,0,0,2


## TEST

In [220]:
submission=pd.read_csv('sample_submission.csv')


In [113]:
#test 데이터 전처리
test_text=list(test['first'])
ids=list(test['id'])

In [114]:
test_features=vectorizer.transform(test_text)

In [138]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)
#results[np.where(np.max(results_proba, axis=1) < 0.60)] = 7

In [146]:
results[np.where((np.max(results_proba, axis=1)<0.6) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.6) & (results == 1))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.6) & (results == 2))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.9) & (results == 3))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.6) & (results == 4))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.9) & (results == 5))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.6) & (results == 6))[0]]=7

In [217]:
submission['level']=results


In [221]:
submission['level']=kip

In [219]:
kip = submission.level

In [222]:
submission.to_csv("baseline(new_pre0.5.csv",index =False)

In [223]:
submission.level.value_counts()

0    1002613
1     396556
3      12419
5       5942
7       1295
4         34
2         34
6         23
Name: level, dtype: int64

In [51]:
submission = pd.DataFrame(submission)
submission['level'].count_values()

AttributeError: 'Series' object has no attribute 'count_values'

In [ ]:
for i in range(6,8):
    print("============ train %d 번 등급 ============="%i)
    print(submission[submission['level'] == i]['first'].value_counts())


In [ ]:
a = pd.read_csv("baseline_2.csv")

In [ ]:
a['level'].value_counts()

In [216]:
submission['level'].value_counts()

0    1002263
1     396341
3      12403
5       5926
7       1892
4         34
2         34
6         23
Name: level, dtype: int64

In [ ]:
0    1002975
1     396368
3      12024
7       6771
5        778

# 최종 학습

In [164]:
#test 데이터 전처리
test_text=list(test['pre_log'])
ids=list(test['id'])

In [165]:
vectorizer=CountVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)

In [166]:
test_features=vectorizer.transform(test_text)

In [167]:
forest=RandomForestClassifier(n_estimators=100)
forest.fit(train_features,train_level)

RandomForestClassifier()

In [207]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)
results[np.where(np.max(results_proba, axis=1) < 0.5)] = 7

results = pd.Series(results)
results.iloc[auto_index] = 7

In [118]:
auto_index

NameError: name 'auto_index' is not defined

In [96]:
vectorizer.get_feature_names()

['__',
 '__init__',
 '__kzf',
 '_a',
 '_ai',
 '_all',
 '_archive',
 '_attachlock',
 '_b',
 '_be',
 '_benchmark_v',
 '_bulk',
 '_c',
 '_cat',
 '_cfg',
 '_checkpoint',
 '_cluster',
 '_codegen',
 '_communications_',
 '_conf_t',
 '_count',
 '_cs',
 '_csv',
 '_custom',
 '_cw',
 '_deadlock',
 '_decoders',
 '_dg',
 '_doc',
 '_dump',
 '_dxp',
 '_emit',
 '_energy_perf_policy',
 '_ey_ghcbalm',
 '_eyhghcbalm',
 '_from',
 '_g',
 '_gdm_',
 '_getconfig',
 '_getfacl',
 '_getsysver',
 '_gpio',
 '_gxcbalm',
 '_header',
 '_hotbackup',
 '_hours',
 '_http_client',
 '_i',
 '_icc_profile',
 '_id',
 '_ii',
 '_index',
 '_ioctl',
 '_is',
 '_itanium',
 '_j',
 '_kdzghcbyokothzhpai',
 '_key',
 '_l',
 '_legacy',
 '_len',
 '_linux',
 '_linux_xen',
 '_load',
 '_local',
 '_locktimeoutid',
 '_log',
 '_lpk',
 '_m',
 '_master',
 '_msearch',
 '_na_',
 '_nav',
 '_neutral_neutral_cw',
 '_ni',
 '_nkincb',
 '_nodes',
 '_notifier',
 '_nxs',
 '_oh',
 '_ondestroy',
 '_ondisk',
 '_onstatuschanged',
 '_open',
 '_os',
 '_p',
 '_pb

In [224]:
auto = pd.read_csv("last_type1.csv")

In [225]:
auto['predict'].value_counts()

0    350415
1       154
Name: predict, dtype: int64

In [226]:
auto_list = list(auto[auto['predict'] == 1]['id'])

In [227]:
idx = submission[submission['id'].isin(auto_list)].index

In [126]:
submission.iloc[idx] = 7

In [228]:
submission[submission['id'].isin(auto_list)]['level'].value_counts()


1    154
Name: level, dtype: int64